## Using WOfS Bitmasking on Waterbodies

* **Product used:**
[wofs_ls](https://explorer.digitalearth.africa/wofs_ls)

## Description 
This notebook uses individual classified images from the Water Observation Feature Layers (WOFLs), which are used in the image masking process. 

The data within a WOFL is stored as a bit field, which is a binary number. Meaning that each digit of a number is independantly set or not based on the presence (1) or absence (0) of a particular attribute (water, cloud, cloud shadow etc). A single decimal value is associated to each pixel, which then provides information on the features within that pixel. 

This notebook works through:

1. Loading WOFL data for a given waterbody and timeframe
2. Creating pixel masks with varying factors 

### Load packages and apps 

This notebook works using two functions, referred to as apps:
` load_waterbody_data` and `pixel_mask`. 

In [ ]:
%matplotlib widget

import warnings

import ipywidgets as widgets
from ipyleaflet import Map, basemaps, Polygon

import geopandas as gpd
import matplotlib.pyplot as plt
import datacube
import sys

from datacube.storage import masking

from deafrica_tools.plotting import rgb
from deafrica_tools.datahandling import load_ard
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.datahandling import wofs_fuser, mostcommon_crs

### Load the waterbody data

The `(load_waterbody_data)` performs several steps including:

* identifies geojson file with the waterbody coordinates
* identifies a custom envelope boundary for the chosen waterbody
* loads WOFLs data 
* returns the data for analysis 

The function takes three arguements: `waterbody_name`, `start_date`, and `end_date`. These arguements determine the named polygon which the function loads, and the timeframe in which the user seeks to find data.

In [ ]:
# The function needs to be defined with the inputs including the name of the waterbody, start date, end date
def load_waterbody_data(waterbody_name, start_date, end_date, file_name, column):
    
    # import the datacube 
    dc = datacube.Datacube(app='pond_analysis')
    
    # set the data frame from the geojson for waterbody coords
    ponds_gdf = gpd.read_file(file_name)
    
    single_pond = ponds_gdf.loc[ponds_gdf[column]== waterbody_name]
    
    # define the coords for the waterbody of choice, including the envelope with the boundaries
    single_pond['geometry'] = single_pond.envelope
    
    bounds = single_pond.bounds
    
    # set up the query
    left, bottom, right, top = bounds.values[0]
    x_coords = (left, right)
    y_coords = (top, bottom)

    time = (start_date, end_date)

    desired_crs = 'EPSG:6933'
    res = 30
    # Construct the data cube query 
    query = {
        'x': x_coords,
        'y': y_coords,
        'time' : time,
        'resolution' : (-res, res)
    }
    
    # load the data 
#     output_crs = mostcommon_crs(dc=dc, product='wofs_ls', query=query)
    wofls = dc.load(
        product = 'wofs_ls',
        group_by="solar_day",
        fuse_func=wofs_fuser, 
        output_crs = desired_crs,
        **query)
    
    # return the data
    return(wofls)


### Pixel Masking

This function converts the WOFL bit field into a binary array, containing True and False values. The `(pixel_mask)` function creates masks using flag labels such as "wet", "dry", "cloud", "cloud shadow" or "no data" to identify: 

* pixels with water prestent
* pixels without water present
* pixels which are cloud covered 
* pixels which hold no data

Then returns the data for analysis. 

In [ ]:
def pixel_mask(data, file_name) :

    # calculate wet and dry attributes
    data['wet'] = masking.make_mask(data, wet=True).water
    data['dry'] = masking.make_mask(data, dry=True).water
    
    # combining flags - clouds
    # cloud (no clouds and shadow)
    data['cloud'] = masking.make_mask(data, cloud=True).water
    data['cloud_shadow'] = masking.make_mask(data, cloud_shadow=True).water
    
    # no data
    data['nodata'] = masking.make_mask(data, nodata = True).water
    
    ponds_gdf = gpd.read_file(file_name)
    pond_shape_raster = xr_rasterize(ponds_gdf, data.wet.isel(time=0), crs='EPSG:6933')
    
    pond_only_data = data.where(pond_shape_raster)
    
    return(pond_only_data)

### Determining Pixel Percent Coverage

The `(percent_coverage)` function calculates the total number of pixels within the set parameters, then computes the percentage of pixels using the flag labels from the previous function. This function then calculates the total percentage of unusable data, which includes "cloud", "cloud_shadow" and "bad data" flags. Allowing for the unusable data to be filtered out of the dataset. 


In [ ]:
def percent_coverage(data):
    
    number_of_pixels = data.isel(time=0).water.count(dim=['x', 'y']).values
    data['water_percent'] = data.wet.sum(dim=['x', 'y'])/number_of_pixels*100
    data['dry_percent'] = data.dry.sum(dim=['x', 'y'])/number_of_pixels*100
    data['cloud_percent'] = data.cloud.sum(dim=['x', 'y'])/number_of_pixels*100
    data['cloud_shadow_percent'] = data.cloud_shadow.sum(dim=['x', 'y'])/number_of_pixels*100
    data['nodata_percent'] = data.nodata.sum(dim=['x', 'y'])/number_of_pixels*100
    
    data['bad_data_percent'] = data.cloud_percent+data.cloud_shadow_percent+data.nodata_percent
    
    # filter observations based on number of no data pixels
    data_filtered = data[['water_percent', 'dry_percent', 'bad_data_percent']].where(data.bad_data_percent<0.05, drop=True)
    
    #calculate the area of the waterbody polygon
    resolution = 30
    m_per_km = 1000
    area_per_pixel = resolution**2/m_per_km**2
    waterbody_area = number_of_pixels*area_per_pixel
    
    
    return(data_filtered, waterbody_area)

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

def add_log(rows, msg):
    max_rows = 3
    rows.append(msg)
    if len(rows) > max_rows:
        rows.pop(0)
    return '<h4>Activity log</h4><ul>{}</ul>'.format('<li>'.join([''] + rows))
    
class waterbody_app(widgets.HBox):
     
    def __init__(self):
        super().__init__()
        
        ##################################################################################################

        # set any initial attributes here 
        self.startdate = '2019-01-01'
        self.enddate = '2019-03-01'
        self.file_name = 'map (2).geojson'
        self.column = 'location'
        
        # load the waterbody gdf
        self.ponds_gdf = gpd.read_file(self.file_name)
        ponds_names = list(self.ponds_gdf[self.column].values)
        self.waterbody = ponds_names[0]
        
        # Find the centroids -- catch warning related to calculating centroids in lat long coords
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            
            self.ponds_gdf['centres'] = self.ponds_gdf['geometry'].centroid
            map_y = self.ponds_gdf.loc[self.ponds_gdf[self.column] == self.waterbody].centres.values[0].y
            map_x = self.ponds_gdf.loc[self.ponds_gdf[self.column] == self.waterbody].centres.values[0].x
       
        ##################################################################################################
    
        # Create the figure        
        self.output = widgets.Output()
        self.output.layout = make_box_layout()
        
        ##################################################################################################
    
        # Create the log
        self.process_log = widgets.HTML(
            value='Press button to run',
            layout= make_box_layout()
        )
        
        ##################################################################################################
    
        # Create the map
        self.m = Map(center=(map_x, map_y), zoom = 12, basemap = basemaps.OpenStreetMap.Mapnik)
        # Draw a polygon for each waterbody on the map.
        for waterbody in ponds_names :
            poly_location = list(self.ponds_gdf.loc[self.ponds_gdf[self.column] == waterbody].geometry.values[0].exterior.coords)
            polygon = Polygon(
                locations = poly_location,
                color ="blue",
                fill_color = "blue"
            )
            self.m.add_layer(polygon)
        self.m.layout = make_box_layout()
        
        ##################################################################################################

        # Create the start date widget
        startdate_picker = widgets.DatePicker(
            description='Pick a start date',
            disabled=False
        )
        
        ##################################################################################################
        
        # Create the end date widget
        enddate_picker = widgets.DatePicker(
            description='Pick an end date',
            disabled=False
        )
        
        ##################################################################################################
        
        # Create the waterbody name widget
        waterbody_name = widgets.Dropdown(
            options = ponds_names,
            value = ponds_names[0],
            description='Waterbody Name:',
            disabled=False,
        )
        
        ##################################################################################################
        
        # Create the run button widget
        run_button = widgets.Button(
            description = 'Run',
        )
        
        ##################################################################################################
        
        # Combine all controls into a single VBox layout and style
        controls = widgets.VBox([
            startdate_picker,
            enddate_picker,
            waterbody_name,
            run_button,
        ])
        controls.layout = make_box_layout()
         
        ##################################################################################################
        
        # Combine all layouts into a single AppLayout
        app_layout = widgets.AppLayout(
            header = None,
            left_sidebar = controls,
            center = self.m,
            right_sidebar = self.output,
            footer = self.process_log,
        )

        ##################################################################################################

        # Display using HBox children attribute
        self.children = [app_layout]
        
        ##################################################################################################
        
        # Run update functions whenever various widgets are changed.
        startdate_picker.observe(self.update_startdate, 'value')
        enddate_picker.observe(self.update_enddate, 'value')
        waterbody_name.observe(self.update_waterbody, 'value')
        waterbody_name.observe(self.update_map, 'value')
        run_button.on_click(self.run_app)
        
        ##################################################################################################
            
    # set the start date to the new edited date
    def update_startdate(self, change):
        self.startdate = change.new
        
    # set the end date to the new edited date    
    def update_enddate(self, change):
        self.enddate = change.new
        
    # set the waterbody to the waterbody chosen in the dropdown
    def update_waterbody(self, change):
        self.waterbody = change.new

    # set the map centre to the centre coordinates of the waterbody chosen in the dropdown
    def update_map(self, change):
        self.waterbody = change.new
        map_y = self.ponds_gdf.loc[self.ponds_gdf[self.column] == self.waterbody].centres.values[0].y
        map_x = self.ponds_gdf.loc[self.ponds_gdf[self.column] == self.waterbody].centres.values[0].x
        self.m.center = (map_x, map_y)
    
    # takes current state of all inputs, runs the data and makes plot when run button is pressed
    def run_app(self, change):
        
        # update the log to show data is being loaded 
        log = []
        self.process_log.value = add_log(log, f'Loading data for {self.waterbody}')
        data = load_waterbody_data(self.waterbody, self.startdate, self.enddate, self.file_name, self.column)
        
        # update the log to show the number of timesteps in loaded data       
        time_step = data.dims['time']
        self.process_log.value = add_log(log, f'Number of loaded timesteps: {time_step}')
        
        mask_data = pixel_mask(data, self.file_name)
        
        percentage_data, water_area = percent_coverage(mask_data)

        #update the log to show the waterbody area 
        self.process_log.value = add_log(log, f'Waterbody coverage area : {water_area}km^2')
        
        with self.output:
            
            self.output.clear_output()
            
            fig, ax = plt.subplots(constrained_layout=True, figsize=(5, 3.5))
            fig.canvas.toolbar_position = 'bottom'

            percentage_data.water_percent.plot(ax = ax, label = 'Water')
            #percentage_data.dry_percent.plot(ax = self.ax, label = 'Dry')
            #percentage_data.bad_data_percent.plot(ax = self.ax, label = 'Unobserved')
            ax.set_title(self.waterbody)
            ax.set_xlabel('Date')
            ax.set_ylabel('Percent Coverage')
            #self.ax.legend(loc = 'upper right')
            ax.set_ylim(0, 100)
            plt.show()
        
        

In [ ]:
waterbody_app()